In [1]:
config = {
    'db': {
        'hostname': 'localhost',
        'port': 8086,
        'username': 'root',
        'password': 'root',
        'database': 'historical-poloniex'
    }
}

We are using [historical poloniex influxdb](https://bitbucket.org/peakrider/poloniex-ticker-history).

In [2]:
from influxdb import InfluxDBClient
client = InfluxDBClient(config['db']['hostname'], 
                        config['db']['port'], 
                        config['db']['username'], 
                        config['db']['password'], 
                        config['db']['database'])

In [3]:
class CoinStore(object):
    def __init__(self, influx_client):
        self.client = client
    def see_available_coins (self, time):
        q ='''
        select * from ticker
        where time <= '{!s}' and time > '{!s}' - 1d
        group by coin
        order by time desc 
        limit 1
        '''.format(time, time)
        result_set = client.query(q)
        coin_generator_tuples = { r[0][1]['coin']: list(r[1])[0]  
                                 for r in result_set.items() }
        return coin_generator_tuples


In [4]:
class Balances(object):
    # balances = { 'COIN': coin_amt }
    def __init__(self, time, balances):
        self.balances = balances
        self.time     = time
        
    def __getitem__ (self, key):
        return self.balances[key]
    
    def held_coins (self):
        return [k  for k in self.balances.keys() 
                if self.balances[k] > 0 ]
    
    def to_influx_point (self):
        return {
            'measurement': 'balance',
            'time': self.time,
            'fields': self.balances,
        }

    # self, Purchase -> Balance
    def apply_purchases (self, time, purchases, fiat='BTC'):
        new_balances = self.balances.copy()
        total_fiat_investment = sum([p.investment_fiat 
                                     for p in purchases])
        new_balances[fiat] = self.balances[fiat] - total_fiat_investment
        for coin, amount, _ in purchases:
            if coin not in new_balances:
                new_balances[coin] = 0
            new_balances[coin] += amount
        return Balances(time, new_balances)

    def estimate_value (self, charts):
        remove = []
        value = 0
        for coin, amount_held in self.balances.items():
            try:
                value += charts[coin]['close'] * amount_held
            except KeyError:
                print('Cannot find a price for ', coin)
                remove.append(coin)
        for removal in remove:
            self.balances.pop(removal)
        return value

    # Balances.from_poloniex(poloniex_client)
    def from_poloniex(poloniex_client):
        # to implement
        pass
    

In [6]:
from collections import namedtuple
Trade = namedtuple('Trade', ['coin', 'investment_fiat'])
Purchase = namedtuple('Purchase', ['coin', 'amount', 'investment_fiat'])

b = Balances('2017-01-01', {'BTC': 1})
p = [Purchase('DOGE', 10, 0.1)]
b.apply_purchases('2017-01-02', p)

In [7]:
class BalancesStore (object):

    def __init__ (self, influx_client):
        self.client = influx_client
    
    def add (self, balances):
        self.client.write_points([
           balances.to_influx_point() 
        ])


In [8]:
cs = CoinStore(client)
test_time = '2017-01-01'
charts = cs.see_available_coins(test_time)
charts

{'1CR': {'close': 0.000301019,
  'high': None,
  'low': None,
  'open': None,
  'price_usd': 0.286883,
  'quoteVolume': None,
  'time': '2016-12-31T13:49:26Z',
  'volume': 5.138296946838955,
  'weightedAverage': None},
 'AMP': {'close': 4.125e-05,
  'high': 4.125e-05,
  'low': 4.058e-05,
  'open': 4.058e-05,
  'price_usd': 0.039003299840000005,
  'quoteVolume': 751.19100022,
  'time': '2017-01-01T00:00:00Z',
  'volume': 0.03077565,
  'weightedAverage': 4.096e-05},
 'ARDR': {'close': 1.007e-05,
  'high': 1.007e-05,
  'low': 1.007e-05,
  'open': 1.007e-05,
  'price_usd': 0.00958894603,
  'quoteVolume': 13.10088863,
  'time': '2017-01-01T00:00:00Z',
  'volume': 0.00013192,
  'weightedAverage': 1.007e-05},
 'BCN': {'close': 5e-08,
  'high': 5e-08,
  'low': 5e-08,
  'open': 5e-08,
  'price_usd': 4.761145e-05,
  'quoteVolume': 0,
  'time': '2017-01-01T00:00:00Z',
  'volume': 0,
  'weightedAverage': 5e-08},
 'BCY': {'close': 0.00013048,
  'high': 0.00013048,
  'low': 0.00013048,
  'open': 0.0

In [9]:

class Strategy (object):
    
    def __init__ (self, coinstore, initial_balances, start_time, end_time, 
                  fiat='BTC'):
        self.start_time = start_time
        self.end_time = end_time
        self.fiat = fiat
        self.coinstore = coinstore
        self.balances = initial_balances
        
    def step (self, time):
        charts = self.coinstore.see_available_coins(time)
        trades = self.buy_and_hold(charts, self.balances)
        purchases = [self.get_purchase_amounts(charts, trade) for trade in trades]
        self.balances = self.balances.apply_purchases(time, purchases)
        value = self.balances.estimate_value(charts)
        return self.balances, value

    def buy_and_hold (self, current_chart_data, current_balances, fiat='BTC'):
        if current_balances.held_coins() == [fiat]:
            # buy some stuffj
            available_coins = current_chart_data.keys()
            amount_to_invest_per_coin = current_balances[fiat] / ( len(available_coins) + 1 )
            trades = [ Trade(c, amount_to_invest_per_coin) 
                      for c in available_coins if c != fiat ]
            return trades
        # if we hold things other than BTC, hold
        # hold
        return []
    
    def get_purchase_amounts (self, current_chart_data, trade,
                          fee=0.0025):
        coin_close = current_chart_data[trade.coin]['close']
        investment = trade.investment_fiat - (trade.investment_fiat * fee)
        amount = investment / coin_close 
        return Purchase(trade.coin, amount, trade.investment_fiat)



In [10]:
import pandas as pd
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

ex_start = date(2017, 3, 1)
ex_end   = date(2017, 5, 1)

strat = Strategy(
    CoinStore(client),
    Balances(ex_start, {'BTC': 1}),
    ex_start, ex_end,
)

for date in daterange(ex_start, ex_end):
    bal, val = strat.step(date)
    print(val)


0.9975769230769264
0.9599300861168985
0.9405165213904021
0.9368310254145771
0.9568252644073861
0.9698906315077955
0.9872696600912488
1.01108336989782
1.0171132982899933
0.9949668220004273
1.0113874304707131
1.0464591989661256
1.076156447729286
1.1785568165344997
1.1910288151040167
1.2447973090223279
1.3449202775979592
1.3600712412588594
1.428506327127613
1.4903156537560365
1.5149861769666093
1.561366899739814
1.601583226422866
1.6963611774888139
1.7615132228483052
1.826343603294726
1.9598435206822242
2.128328864405041
2.327082833679148
2.218349522394537
2.3823878743626965
2.3105927407235565
2.3748086841347638
2.330138858157161
2.2530648003114466
2.331167014580166
2.438779691300572
2.2584217714191306
2.264117256407353
2.3711067993891786
2.3097320712784906
2.4158155788279383
2.4122201669601897
2.5450623032195177
2.608927647282222
2.7312336599050697
2.9280919907465552
3.1670396087363084
2.981873522556867
3.0994874713129597
3.148635608109599
2.949301762236681
3.026517451705114
2.9704974046